In [1]:
# Cell 1: Install Packages
!pip install -q -U nemo_toolkit['asr'] pydub pandas fastapi uvicorn python-multipart pyngrok torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118 wget
!pip install numpy==1.26.4 --force-reinstall
# Add moviepy for video processing
!pip install moviepy

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.5 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 10.8 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Cell 2: Imports and Global Model Loading
import nemo.collections.asr as nemo_asr
import torch
from pathlib import Path
from pydub import AudioSegment
import os
import shutil
import uvicorn
import nest_asyncio
import asyncio
from pyngrok import ngrok, conf
import subprocess # For running wget
import tempfile
from moviepy.editor import VideoFileClip # For video processing

from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse

# --- Global ASR Model Setup ---
print("Loading ASR model globally...")
try:
    asr_model = nemo_asr.models.ASRModel.from_pretrained(model_name="nvidia/parakeet-tdt-0.6b-v2")
    asr_model.eval()
    if torch.cuda.is_available():
        print("Moving model to GPU and using half precision.")
        asr_model.cuda()
        asr_model.half() 
    else:
        print("CUDA not available, model will run on CPU.")
    print("ASR Model loaded successfully.")
except Exception as e:
    print(f"Error loading ASR model: {e}")
    asr_model = None # Set to None if loading fails

# --- Example Audio Configuration ---
EXAMPLE_AUDIO_FILENAME = "example_audio_nemo.wav" # Standard NeMo example
EXAMPLE_AUDIO_DOWNLOAD_URL = "https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav"
KAGGLE_WORKING_DIR = Path("/kaggle/working/")
EXAMPLE_AUDIO_PATH = KAGGLE_WORKING_DIR / EXAMPLE_AUDIO_FILENAME

# --- Directories for API ---
TEMP_UPLOAD_DIR_API = KAGGLE_WORKING_DIR / "api_temp_uploads"
TEMP_UPLOAD_DIR_API.mkdir(parents=True, exist_ok=True)

PROCESSED_AUDIO_DIR_API = KAGGLE_WORKING_DIR / "api_processed_audio"
PROCESSED_AUDIO_DIR_API.mkdir(parents=True, exist_ok=True)

error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evalu

Loading ASR model globally...


parakeet-tdt-0.6b-v2.nemo:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

[NeMo I 2025-05-15 00:05:38 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-05-15 00:05:38 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    skip_missing_manifest_entries: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 2
    pin_memory: true
    max_duration: 40.0
    min_duration: 0.1
    text_field: answer
    batch_duration: null
    use_bucketing: true
    bucket_duration_bins: null
    bucket_batch_size: null
    num_buckets: 30
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    
[NeMo W 2025-05-15 00:05:38 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config :

[NeMo I 2025-05-15 00:05:38 nemo_logging:393] PADDING: 0
[NeMo I 2025-05-15 00:05:45 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-05-15 00:05:45 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-05-15 00:05:45 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-05-15 00:05:48 nemo_logging:393] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--parakeet-tdt-0.6b-v2/snapshots/50aec6a056e85b9f95b612df08a2bddc55b58714/parakeet-tdt-0.6b-v2.nemo.
Moving model to GPU and using half precision.
ASR Model loaded successfully.


In [8]:
# Cell 3: Download Example Audio File
if asr_model is not None: # Only proceed if model loaded
    print(f"Checking for example audio: {EXAMPLE_AUDIO_PATH}")
    if not EXAMPLE_AUDIO_PATH.exists() or EXAMPLE_AUDIO_PATH.stat().st_size == 0:
        print(f"Downloading example audio to {EXAMPLE_AUDIO_PATH}...")
        try:
            subprocess.run(["wget", "-O", str(EXAMPLE_AUDIO_PATH), EXAMPLE_AUDIO_DOWNLOAD_URL], check=True, capture_output=True, text=True)
            print("Download complete.")
            if EXAMPLE_AUDIO_PATH.exists() and EXAMPLE_AUDIO_PATH.stat().st_size > 0:
                print(f"Example audio file {EXAMPLE_AUDIO_PATH} is present and not empty.")
            else:
                print(f"Error: Example audio file {EXAMPLE_AUDIO_PATH} was not downloaded correctly or is empty after wget attempt.")
        except subprocess.CalledProcessError as e:
            print(f"Error downloading example audio with wget: {e}")
            print(f"stderr: {e.stderr}")
        except FileNotFoundError:
            print("Error: wget command not found. Please ensure wget is installed in the Kaggle environment.")
        except Exception as e_download:
            print(f"An unexpected error occurred during download: {e_download}")
    else:
        print(f"Example audio file {EXAMPLE_AUDIO_PATH} already exists.")
else:
    print("ASR model did not load, skipping example audio download.")

Checking for example audio: /kaggle/working/example_audio_nemo.wav
Example audio file /kaggle/working/example_audio_nemo.wav already exists.


In [9]:
# Cell 4: Audio Preprocessing Function
def preprocess_audio_for_nemo(input_audio_path_str: str, output_processing_dir: Path) -> str:
    """
    Preprocesses an audio file to be 16kHz mono WAV for NeMo.
    Saves the processed file in output_processing_dir.
    Returns the path (as a string) to the processed file.
    """
    print(f"Starting preprocessing for: {input_audio_path_str}")
    audio_path_obj = Path(input_audio_path_str)
    
    if not audio_path_obj.exists():
        print(f"Error: Input audio file not found at {input_audio_path_str}")
        raise FileNotFoundError(f"Input audio file not found: {input_audio_path_str}")

    try:
        audio = AudioSegment.from_file(input_audio_path_str)
        print(f"Original audio - Channels: {audio.channels}, Frame Rate: {audio.frame_rate}, Duration: {audio.duration_seconds:.2f}s")
    except Exception as e:
        print(f"Error loading audio file {input_audio_path_str} with pydub: {e}")
        raise
    
    resampled = False
    mono = False
    needs_export = False

    target_sr = 16000
    if audio.frame_rate != target_sr:
        print(f"Resampling audio from {audio.frame_rate}Hz to {target_sr}Hz")
        audio = audio.set_frame_rate(target_sr)
        resampled = True
        needs_export = True

    if audio.channels != 1:
        print(f"Converting to mono (from {audio.channels} channels)")
        audio = audio.set_channels(1)
        mono = True
        needs_export = True

    # Define the path for the processed file
    # Use a consistent naming convention for the processed file in the specific output directory
    processed_filename = f"{audio_path_obj.stem}_processed_16kHz_mono.wav"
    processed_path_str = str(output_processing_dir / processed_filename)

    if needs_export:
        try:
            audio.export(processed_path_str, format="wav")
            print(f"Processed audio saved to: {processed_path_str}")
        except Exception as e:
            print(f"Error exporting processed audio to {processed_path_str}: {e}")
            raise
        return processed_path_str
    else:
        # If no processing was strictly needed but we want to ensure it's in the target dir
        # and has the standard processed name, copy it.
        if str(audio_path_obj.resolve()) != str(Path(processed_path_str).resolve()):
            print(f"Audio was already 16kHz mono. Copying to target processing directory: {processed_path_str}")
            shutil.copy(input_audio_path_str, processed_path_str)
            return processed_path_str
        else:
            print("Audio already 16kHz mono and in the correct location/name.")
            return input_audio_path_str

# Function to extract audio from video
def extract_audio_from_video(video_path: str, output_dir: Path) -> str:
    """
    Extracts audio from a video file and saves it as WAV.
    Returns the path to the extracted audio file.
    """
    video_path_obj = Path(video_path)
    output_audio_path = str(output_dir / f"{video_path_obj.stem}_audio.wav")
    
    try:
        print(f"Extracting audio from video: {video_path}")
        video = VideoFileClip(video_path)
        audio = video.audio
        if audio is None:
            raise ValueError("No audio stream found in the video file")
            
        print(f"Saving extracted audio to: {output_audio_path}")
        audio.write_audiofile(output_audio_path, fps=16000, nbytes=2, codec='pcm_s16le')
        video.close()
        
        return output_audio_path
    except Exception as e:
        print(f"Error extracting audio from video: {e}")
        raise

In [15]:
# Cell 5: FastAPI Application with Video Processing Endpoints
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*", "http://localhost:3000", "https://localhost:3000", "http://localhost", "http://127.0.0.1:3000"],
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
    expose_headers=["*"], # Expose all headers
)

# Keep original example endpoint for backward compatibility
@app.post("/transcribe_example_audio/")
async def transcribe_example_audio_endpoint(uploaded_file: UploadFile = File(None)): # uploaded_file is ignored
    if asr_model is None:
        print("Error: ASR model is not available (was not loaded in Cell 2).")
        raise HTTPException(status_code=503, detail="Transcription service is currently unavailable (model not loaded).")

    if not EXAMPLE_AUDIO_PATH.exists() or EXAMPLE_AUDIO_PATH.stat().st_size == 0:
        print(f"Critical Error: Example audio file {EXAMPLE_AUDIO_PATH} not found or empty on server. Cell 3 (download) might have failed.")
        raise HTTPException(status_code=500, detail="Server configuration error: Example audio file is missing.")

    processed_example_audio_path_str = None
    try:
        # Preprocess the *example* audio file (ensures it's in the right format for NeMo)
        print(f"Processing example audio: {EXAMPLE_AUDIO_PATH}")
        processed_example_audio_path_str = preprocess_audio_for_nemo(str(EXAMPLE_AUDIO_PATH), PROCESSED_AUDIO_DIR_API)
        
        # Ensure model is on the correct device and precision for transcription
        if torch.cuda.is_available():
            # Ensure model parameters are on CUDA device
            if not next(asr_model.parameters()).is_cuda:
                print("Model parameters were not on CUDA inside endpoint. Moving model to CUDA.")
                asr_model.cuda() # This moves the entire model, including parameters
            
            # Check current dtype of model parameters and apply .half() if not already float16
            current_param_tensor = next(asr_model.parameters()).data # Get the underlying tensor data
            if current_param_tensor.dtype != torch.float16:
                if hasattr(asr_model, 'half') and callable(asr_model.half):
                    print(f"Model on CUDA but parameters dtype is {current_param_tensor.dtype}. Applying .half() for transcription.")
                    asr_model.half()
                else:
                    print(f"Model on CUDA but parameters dtype is {current_param_tensor.dtype}, and .half() method is not available/callable.")
            else:
                print("Model is already on CUDA and parameters are in float16 precision.")
        else:
            # Optional: Ensure model is on CPU if CUDA is not available (defensive)
            if next(asr_model.parameters()).is_cuda: 
                print("CUDA reported as not available, but model parameters are on CUDA. Moving to CPU.")
                asr_model.cpu() 
            print("Running transcription on CPU.")
        
        print(f"Transcribing processed example audio: {processed_example_audio_path_str}...")
        # Use timestamps=True, as confirmed by your original working Colab notebook
        hypotheses = asr_model.transcribe([processed_example_audio_path_str], timestamps=True)

        if not hypotheses or not isinstance(hypotheses, list) or not hypotheses[0]:
            print("Transcription of example audio failed (no hypotheses returned from model.transcribe).")
            raise HTTPException(status_code=500, detail="Transcription of example audio failed (no hypotheses).")

        first_hypothesis = hypotheses[0]

        # Based on your original Colab: output[0].timestamp['segment']
        if not hasattr(first_hypothesis, 'timestamp') or 'segment' not in first_hypothesis.timestamp:
            print("Example audio transcription output format error: 'timestamp' or 'segment' key missing in hypothesis.")
            # Log details for debugging
            print(f"Hypothesis object dir: {dir(first_hypothesis)}")
            if hasattr(first_hypothesis, 'text'): print(f"Hypothesis text (if available): {first_hypothesis.text}")
            if hasattr(first_hypothesis, 'timestamp'): 
                ts_attr = first_hypothesis.timestamp
                print(f"Hypothesis timestamp attribute type: {type(ts_attr)}")
                if isinstance(ts_attr, dict):
                    print(f"Hypothesis timestamp keys: {ts_attr.keys()}")
                else:
                    print(f"Hypothesis timestamp attribute is not a dict.")
            else:
                print("Hypothesis does not have a 'timestamp' attribute.")
            raise HTTPException(status_code=500, detail="Internal server error: Unexpected transcription output format for example audio.")

        segment_data = first_hypothesis.timestamp['segment']

        if not segment_data: 
            print("No segments found in example audio transcription (audio might be silent or very short).")
            return JSONResponse(content={"filename": "example_transcription.txt", "transcription_text": ""})

        transcript_lines = []
        # Keys from your original Colab example: 'start', 'end', 'segment'
        for s_data_dict in segment_data:
            if not isinstance(s_data_dict, dict) or not all(k in s_data_dict for k in ['start', 'end', 'segment']):
                print(f"Segment data dictionary malformed or missing expected keys: {s_data_dict}")
                continue 
            start_time = s_data_dict['start']
            end_time = s_data_dict['end']
            text_segment = s_data_dict['segment']
            transcript_lines.append(f"{start_time:.2f}s - {end_time:.2f}s : {text_segment}")
        
        final_transcript_text = "\n".join(transcript_lines)
        
        print("Example audio transcription complete.")
        return JSONResponse(content={"filename": "example_transcription.txt", "transcription_text": final_transcript_text})

    except FileNotFoundError as e:
        print(f"File not found during API processing (likely preprocess_audio_for_nemo failed to find input): {e}")
        raise HTTPException(status_code=500, detail=f"Server error (file not found): {e}")
    except Exception as e:
        print(f"Error during example audio transcription endpoint: {e}")
        import traceback
        traceback.print_exc() # This will print the full traceback to the notebook console
        raise HTTPException(status_code=500, detail=f"An error occurred during transcription: {str(e)}")
    finally:
        # Clean up the processed example audio file (which was created by preprocess_audio_for_nemo)
        if processed_example_audio_path_str and Path(processed_example_audio_path_str).exists():
            try:
                Path(processed_example_audio_path_str).unlink()
                print(f"Removed temporary processed example audio file: {processed_example_audio_path_str}")
            except OSError as e_unlink:
                print(f"Error removing temporary file {processed_example_audio_path_str}: {e_unlink}")
        # Note: The originally uploaded file (if any) is not explicitly handled here as it's ignored by this endpoint.
        # The global ASR model is kept on its device (e.g., GPU) for subsequent requests.


# NEW ENDPOINT: For processing actual video uploads
@app.post("/transcribe_video/")
async def transcribe_video_endpoint(uploaded_file: UploadFile = File(...)):  # The '...' makes it required
    if asr_model is None:
        print("Error: ASR model is not available (was not loaded in Cell 2).")
        raise HTTPException(status_code=503, detail="Transcription service is currently unavailable (model not loaded).")

    # Check for valid file
    if not uploaded_file or not uploaded_file.filename:
        raise HTTPException(status_code=400, detail="No file uploaded")
    
    video_path = None
    audio_path = None
    processed_audio_path = None
    
    try:
        # Create a temporary file to store the uploaded video
        with tempfile.NamedTemporaryFile(delete=False, suffix=Path(uploaded_file.filename).suffix) as tmp:
            video_path = tmp.name
            # Copy uploaded file to temp file
            content = await uploaded_file.read()
            tmp.write(content)
        
        print(f"Video saved temporarily to: {video_path}")
        
        # Extract audio from video
        audio_path = extract_audio_from_video(video_path, TEMP_UPLOAD_DIR_API)
        
        # Preprocess the extracted audio
        processed_audio_path = preprocess_audio_for_nemo(audio_path, PROCESSED_AUDIO_DIR_API)
        
        # Ensure model is on correct device and precision
        if torch.cuda.is_available():
            if not next(asr_model.parameters()).is_cuda:
                asr_model.cuda()
            
            current_param_tensor = next(asr_model.parameters()).data
            if current_param_tensor.dtype != torch.float16:
                if hasattr(asr_model, 'half') and callable(asr_model.half):
                    asr_model.half()
        
        # Transcribe the audio
        print(f"Transcribing video audio: {processed_audio_path}")
        hypotheses = asr_model.transcribe([processed_audio_path], timestamps=True)
        
        if not hypotheses or not isinstance(hypotheses, list) or not hypotheses[0]:
            raise HTTPException(status_code=500, detail="Transcription failed (no results)")
        
        first_hypothesis = hypotheses[0]
        
        if not hasattr(first_hypothesis, 'timestamp') or 'segment' not in first_hypothesis.timestamp:
            print(f"Unexpected transcription output format: {dir(first_hypothesis)}")
            raise HTTPException(status_code=500, detail="Unexpected transcription output format")
        
        segment_data = first_hypothesis.timestamp['segment']
        
        if not segment_data:
            return JSONResponse(content={
                "filename": f"{Path(uploaded_file.filename).stem}_transcription.txt",
                "transcription_text": ""
            })
        
        transcript_lines = []
        for s_data_dict in segment_data:
            if not isinstance(s_data_dict, dict) or not all(k in s_data_dict for k in ['start', 'end', 'segment']):
                continue
            
            start_time = s_data_dict['start']
            end_time = s_data_dict['end']
            text_segment = s_data_dict['segment']
            transcript_lines.append(f"{start_time:.2f}s - {end_time:.2f}s : {text_segment}")
        
        final_transcript_text = "\n".join(transcript_lines)
        
        print("Video transcription complete")
        return JSONResponse(content={
            "filename": f"{Path(uploaded_file.filename).stem}_transcription.txt", 
            "transcription_text": final_transcript_text
        })
    
    except Exception as e:
        print(f"Error during video transcription: {e}")
        import traceback
        traceback.print_exc()
        raise HTTPException(status_code=500, detail=f"An error occurred during transcription: {str(e)}")
    
    finally:
        # Clean up temporary files
        for path in [video_path, audio_path, processed_audio_path]:
            if path and Path(path).exists():
                try:
                    Path(path).unlink()
                    print(f"Removed temporary file: {path}")
                except OSError as e:
                    print(f"Error removing temporary")

In [17]:
# Cell 6: Start Ngrok and Uvicorn Server
NGROK_AUTH_TOKEN = "2x43w3eV98ow7s4Esh2761K7Xib_5piK9Ef22bp5q45ZhT9E3" # <--- REPLACE THIS WITH YOUR ACTUAL NGROK TOKEN

if NGROK_AUTH_TOKEN == "YOUR_NGROK_AUTHTOKEN" or not NGROK_AUTH_TOKEN:
    print("CRITICAL: Please set your NGROK_AUTH_TOKEN in Cell 6.")
elif asr_model is None:
    print("CRITICAL: ASR model failed to load (see Cell 2). API will not start.")
elif not EXAMPLE_AUDIO_PATH.exists() or EXAMPLE_AUDIO_PATH.stat().st_size == 0:
    print(f"CRITICAL: Example audio file {EXAMPLE_AUDIO_PATH} is missing or empty (see Cell 3). API will not start.")
else:
    conf.get_default().auth_token = NGROK_AUTH_TOKEN
    # Kill any existing ngrok tunnels
    try:
        tunnels = ngrok.get_tunnels()
        for tunnel in tunnels:
            ngrok.disconnect(tunnel.public_url)
            print(f"Disconnected previous tunnel: {tunnel.public_url}")
        ngrok.kill()
        print("Killed all ngrok processes.")
    except Exception as e_ngrok_ops:
        print(f"Error managing ngrok processes (this is often okay if none were running): {e_ngrok_ops}")

    PORT = 7860 
    try:
        # Connect to ngrok
        public_url_obj = ngrok.connect(PORT)
        # Make sure we get a clean URL string
        if hasattr(public_url_obj, 'public_url'):
            public_url_str = public_url_obj.public_url
        else:
            public_url_str = str(public_url_obj).replace('NgrokTunnel: "', '').replace('" -> "http://localhost:7860"', '')
        
        # Ensure it's a clean URL string without extra quotes or NgrokTunnel text
        if 'ngrok-free.app' in public_url_str and not public_url_str.startswith('http'):
            public_url_str = f"https://{public_url_str}"

        print(f"Ngrok tunnel active: {public_url_str}")
        example_endpoint = f"{public_url_str}/transcribe_example_audio/"
        video_endpoint = f"{public_url_str}/transcribe_video/"
        
        print(f"Your example transcription API endpoint will be: {example_endpoint}")
        print(f"Your video transcription API endpoint will be: {video_endpoint}")
        print("\n*** IMPORTANT: UPDATE YOUR FRONTEND API.TS WITH THESE ENDPOINTS ***")
        print("In src/services/api.ts, update the fetch URL to:")
        print(f"const response = await fetch('{video_endpoint}', {{")
        
        # Run Uvicorn with a different approach for Jupyter notebooks
        print(f"\nStarting Uvicorn server on port {PORT}...")
        import uvicorn.config
        import asyncio
        import threading
        
        server_config = uvicorn.config.Config(app=app, host="0.0.0.0", port=PORT, log_level="info")
        server = uvicorn.server.Server(config=server_config)
        
        # Run the server in a separate thread to avoid asyncio issues
        def run_server():
            asyncio.set_event_loop(asyncio.new_event_loop())
            server.run()
        
        thread = threading.Thread(target=run_server, daemon=True)
        thread.start()
        
        # Verify the server is running
        import time
        import requests
        print("Waiting for server to start...")
        time.sleep(2)  # Give the server a moment to start
        
        try:
            health_check = requests.get(f"http://localhost:{PORT}")
            print(f"Server health check: {health_check.status_code}")
            print(f"Server is running in background thread. Ngrok tunnel is active.")
            print("The API will continue running until you stop the notebook or kernel.")
            print("\n*** After the server is running, test your endpoints with these commands: ***")
            print(f"curl -X POST {example_endpoint}")
            print(f"curl -X POST {video_endpoint} -F \"uploaded_file=@/path/to/your/video.mp4\"")
        except Exception as e:
            print(f"Server may not be fully started yet: {e}")
            print("Continue with the next steps anyway, the server should start soon.")
        
    except Exception as e_ngrok_run:
        print(f"Could not start ngrok or Uvicorn server: {e_ngrok_run}")
        import traceback
        traceback.print_exc()

Killed all ngrok processes.
Ngrok tunnel active: https://3355-34-168-128-192.ngrok-free.app
Your example transcription API endpoint will be: https://3355-34-168-128-192.ngrok-free.app/transcribe_example_audio/
Your video transcription API endpoint will be: https://3355-34-168-128-192.ngrok-free.app/transcribe_video/

*** IMPORTANT: UPDATE YOUR FRONTEND API.TS WITH THESE ENDPOINTS ***
In src/services/api.ts, update the fetch URL to:
const response = await fetch('https://3355-34-168-128-192.ngrok-free.app/transcribe_video/', {

Starting Uvicorn server on port 7860...


INFO:     Started server process [35]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 7860): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


Waiting for server to start...
INFO:     127.0.0.1:49744 - "GET / HTTP/1.1" 404 Not Found
Server health check: 404
Server is running in background thread. Ngrok tunnel is active.
The API will continue running until you stop the notebook or kernel.

*** After the server is running, test your endpoints with these commands: ***
curl -X POST https://3355-34-168-128-192.ngrok-free.app/transcribe_example_audio/
curl -X POST https://3355-34-168-128-192.ngrok-free.app/transcribe_video/ -F "uploaded_file=@/path/to/your/video.mp4"
INFO:     2803:4600:1111:ee1:2dde:c401:5274:65ef:0 - "POST /transcribe_video/ HTTP/1.1" 422 Unprocessable Entity
Video saved temporarily to: /tmp/tmpwhnjda72.mp4
Extracting audio from video: /tmp/tmpwhnjda72.mp4
Saving extracted audio to: /kaggle/working/api_temp_uploads/tmpwhnjda72_audio.wav
MoviePy - Writing audio in /kaggle/working/api_temp_uploads/tmpwhnjda72_audio.wav


MoviePy - Done.


Starting preprocessing for: /kaggle/working/api_temp_uploads/tmpwhnjda72_audio.wav
Original audio - Channels: 2, Frame Rate: 16000, Duration: 40.32s
Converting to mono (from 2 channels)
Processed audio saved to: /kaggle/working/api_processed_audio/tmpwhnjda72_audio_processed_16kHz_mono.wav
Transcribing video audio: /kaggle/working/api_processed_audio/tmpwhnjda72_audio_processed_16kHz_mono.wav
[NeMo I 2025-05-15 00:25:14 nemo_logging:393] Timestamps requested, setting decoding timestamps to True. Capture them in Hypothesis object,                         with output[0][idx].timestep['word'/'segment'/'char']
[NeMo I 2025-05-15 00:25:14 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}


...

Video transcription complete
Removed temporary file: /tmp/tmpsc1n57_d.mp4
Removed temporary file: /kaggle/working/api_temp_uploads/tmpsc1n57_d_audio.wav
Removed temporary file: /kaggle/working/api_processed_audio/tmpsc1n57_d_audio_processed_16kHz_mono.wav
INFO:     2803:4600:1111:ee1:2dde:c401:5274:65ef:0 - "POST /transcribe_video/ HTTP/1.1" 200 OK


Video saved temporarily to: /tmp/tmpike6djbe.mp4
Extracting audio from video: /tmp/tmpike6djbe.mp4
Saving extracted audio to: /kaggle/working/api_temp_uploads/tmpike6djbe_audio.wav
MoviePy - Writing audio in /kaggle/working/api_temp_uploads/tmpike6djbe_audio.wav


MoviePy - Done.
Starting preprocessing for: /kaggle/working/api_temp_uploads/tmpike6djbe_audio.wav
Original audio - Channels: 2, Frame Rate: 16000, Duration: 22.65s
Converting to mono (from 2 channels)
Processed audio saved to: /kaggle/working/api_processed_audio/tmpike6djbe_audio_processed_16kHz_mono.wav
Transcribing video audio: /kaggle/working/api_processed_audio/tmpike6djbe_audio_processed_16kHz_mono.wav
[NeMo I 2025-05-15 00:32:22 nemo_logging:393] Timestamps requested, setting decoding timestamps to True. Capture them in Hypothesis object,                         with output[0][idx].timestep['word'/'segment'/'char']
[NeMo I 2025-05-15 00:32:22 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}


[NeMo W 2025-05-15 00:32:22 nemo_logging:405] `include_duration` is not implemented for CUDA graphs
Transcribing: 100%|██████████| 1/1 [00:00<00:00,  4.31it/s]

Video transcription complete
Removed temporary file: /tmp/tmpike6djbe.mp4
Removed temporary file: /kaggle/working/api_temp_uploads/tmpike6djbe_audio.wav
Removed temporary file: /kaggle/working/api_processed_audio/tmpike6djbe_audio_processed_16kHz_mono.wav
INFO:     2803:4600:1111:ee1:2dde:c401:5274:65ef:0 - "POST /transcribe_video/ HTTP/1.1" 200 OK
